<a href="https://colab.research.google.com/github/R1tsuko/big_data/blob/main/L2%20-%20Reports%20with%20Apache%20Spark/BigData_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [49]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [148]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
sc = spark.sparkContext
spark

In [4]:
!wget https://archive.org/download/stackexchange-snapshot-2018-03-14/stackoverflow.com-Posts.7z

--2024-04-24 12:41:28--  https://archive.org/download/stackexchange-snapshot-2018-03-14/stackoverflow.com-Posts.7z
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia802306.us.archive.org/33/items/stackexchange-snapshot-2018-03-14/stackoverflow.com-Posts.7z [following]
--2024-04-24 12:41:29--  https://ia802306.us.archive.org/33/items/stackexchange-snapshot-2018-03-14/stackoverflow.com-Posts.7z
Resolving ia802306.us.archive.org (ia802306.us.archive.org)... 207.241.228.46
Connecting to ia802306.us.archive.org (ia802306.us.archive.org)|207.241.228.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12550618543 (12G) [application/x-7z-compressed]
Saving to: ‘stackoverflow.com-Posts.7z’

stackoverflow.com-P 100%[===================>]  11.69G  9.89MB/s    in 67m 53s 

2024-04-24 13:49:22 (2.94 MB/s) - ‘stackoverflow.com-Post

In [5]:
!7z e stackoverflow.com-Posts.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 12550618543 bytes (12 GiB)

Extracting archive: stackoverflow.com-Posts.7z
--
Path = stackoverflow.com-Posts.7z
Type = 7z
Physical Size = 12550618543
Headers Size = 122
Method = BZip2
Solid = -
Blocks = 1

  0%      0% - Posts.xml                  1% - Posts.xml                  2% - Posts.xml                  3% - Posts.xml                  4% - Posts.xml                  5% - Posts.xml                  6% - Posts.xml                  7% - Posts.xml                

In [7]:
from google.colab import files
uploaded = files.upload()

Saving programming-languages.csv to programming-languages.csv


In [136]:
posts_lines = sc.textFile("Posts.xml")

In [149]:
posts_lines.getNumPartitions()

1869

In [57]:
posts_sample = posts_lines.sample(False, 1 / 10000)
posts_sample.count()

4003

In [147]:
import xml.etree.ElementTree as ET
from datetime import datetime

In [153]:
posts_sample_tagged = posts_sample.filter(lambda elem: ET.fromstring(elem).attrib.get('Tags'))

In [146]:
prog_lang = spark.read.csv("programming-languages.csv")
prog_lang_list = [str(x[0]).lower() for x in prog_lang.collect()]
prog_lang_list[0:5]

['name', 'a# .net', 'a# (axiom)', 'a-0 system', 'a+']

In [166]:
def process_xml_string(xml_string):
  try:
    elem = ET.fromstring(xml_string).attrib
  except:
    return [(('', ''), 1)]
  tags = elem.get('Tags')
  res = []
  if not tags:
    return [(('', ''), 1)]
  year = elem.get('CreationDate')[0:4]
  for lang in prog_lang_list:
    if '<' + lang + '>' in tags.lower():
      res.append(((year, lang), 1))
  return res

In [133]:
from google.colab import output
output.serve_kernel_port_as_window(4040, path='/jobs/index.html')
! spark-submit pi.py 300

<IPython.core.display.Javascript object>

/bin/bash: line 1: spark-submit: command not found


In [167]:
from operator import add
all_res = posts_lines.flatMap(process_xml_string).reduceByKey(add).sortBy(lambda x: x[1], ascending=False)

In [169]:
res_list = all_res.collect()

In [173]:
res_list.pop(0) # убирается результат ('', '')
res_list[:5]

[(('2015', 'javascript'), 257159),
 (('2014', 'javascript'), 237517),
 (('2014', 'java'), 219074),
 (('2015', 'java'), 216950),
 (('2017', 'python'), 208917)]

In [188]:
final_res = {}
for year in range(2010, 2018):
  final_res[str(year) + '_language'] = []
  final_res[str(year) + '_mentions'] = []
for row in res_list:
  year, lang, mentions = int(row[0][0]), row[0][1], row[1]
  if year in range(2010, 2018) and len(final_res[str(year) + '_language']) < 10:
    final_res[str(year) + '_language'].append(lang)
    final_res[str(year) + '_mentions'].append(mentions)

In [191]:
from pandas import DataFrame
DataFrame.iteritems = DataFrame.items # https://stackoverflow.com/questions/76404811/attributeerror-dataframe-object-has-no-attribute-iteritems
df = DataFrame(final_res)
ddf = spark.createDataFrame(df)
#ddf.write.parquet('lab3_report.parquet')
ddf

2010_language,2010_mentions,2011_language,2011_mentions,2012_language,2012_mentions,2013_language,2013_mentions,2014_language,2014_mentions,2015_language,2015_mentions,2016_language,2016_mentions,2017_language,2017_mentions
java,54549,java,99538,java,145701,javascript,197182,javascript,237517,javascript,257159,java,198884,python,208917
php,51821,php,96820,javascript,136984,java,192312,java,219074,java,216950,php,164037,java,194824
javascript,43424,javascript,90368,php,133074,php,169524,php,180886,php,173370,python,159577,php,156804
python,27110,python,42330,python,64497,python,96874,python,117016,python,137938,r,44602,r,56012
objective-c,17898,objective-c,39022,objective-c,46996,objective-c,47778,objective-c,48550,r,40844,c,34909,c,31879
c,15323,c,22678,c,31138,c,40907,c,42080,c,40326,ruby,24172,typescript,23657
ruby,10188,ruby,18813,ruby,24498,ruby,29413,r,31008,objective-c,35928,objective-c,23538,ruby,18514
flex,5164,perl,6792,r,12225,r,22335,ruby,30140,ruby,28119,bash,16182,bash,16817
perl,5034,flex,6132,bash,8283,bash,12521,bash,15209,bash,15455,scala,14751,objective-c,14720
delphi,3869,r,5847,perl,7807,matlab,11746,matlab,14150,matlab,13972,typescript,13573,scala,14658
